In [ ]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [ ]:
!nvidia-smi

Wed May 18 01:54:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8    10W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers
!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/pytorch_weights.tar.gz 
!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/vocab.txt 
!wget https://users.dcc.uchile.cl/~jperez/beto/cased_2M/config.json 
!tar -xzvf pytorch_weights.tar.gz
!mv config.json pytorch/.
!mv vocab.txt pytorch/.

     |████████████████████████████████| 4.2 MB 23.7 MB/s 
     |████████████████████████████████| 6.6 MB 57.8 MB/s 
     |████████████████████████████████| 596 kB 68.5 MB/s 
     |████████████████████████████████| 84 kB 4.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
--2022-05-18 01:54:33--  https://users.dcc.uchile.cl/~jperez/beto/cased_2M/pytorch_weights.tar.gz
Resolving users.dcc.uchile.cl (users.dcc.uchile.cl)... 200.9.99.211, 192.80.24.4
Connecting to users.dcc.uchile.cl (users.dcc.uchile.cl)|200.9.99.211|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 409871727 (391M) [application/x-gzip]
Saving to: ‘pytorch_weights.tar.gz’

pytorch_weights.tar 100%[===================>] 390.88M  7.41MB/s    in 52s     

2022-05-18 01:55:25 (7.50 MB/s) - ‘pytorch_weights.tar.gz’ saved [409871727/409871727]

--2022-05-18 01:55:26--  https://users.dcc.uchile.cl/~j

In [ ]:
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import shutil
import sys   
from sklearn.model_selection import train_test_split

In [ ]:
train_path = '/content/drive/MyDrive/Recuperacion de la informacion/BERT/translatedEs.csv'

In [ ]:
df = pd.read_csv(train_path)
train_df, test_df = train_test_split(df, test_size=0.1)
train_df.shape, test_df.shape

((9000, 13), (1000, 13))

In [ ]:
train_df.head()

,ID,Tweet,Optimistic,Thankful,Empathetic,Pessimistic,Anxious,Sad,Annoyed,Denial,Official report,Surprise,Joking
8863,1.245250e+18,Un puto tren por hora y todavía quiere ser put...,0,0,0,0,0,0,1,0,0,0,0
1889,1.245158e+18,Abril deseo: ser mejor dada la pérdida saludab...,1,0,1,0,0,0,0,0,0,0,0
3276,1.245175e+18,hijo de puta mi salud mental nunca ha sido tan...,0,0,0,0,1,0,1,0,0,0,0
2582,1.245166e+18,"Dios por la subasta, pero si la Corona empeora...",0,0,0,0,1,0,0,0,0,0,0
240,1.245141e+18,Se debería haber tenido una carta si theyve un...,0,0,1,0,0,0,0,0,0,0,1


In [ ]:
# dropping useless features/columns
train_df.drop(labels=['ID'], axis=1, inplace=True)

In [ ]:
train_df.columns

Index(['Tweet', 'Optimistic', 'Thankful', 'Empathetic', 'Pessimistic',
       'Anxious', 'Sad', 'Annoyed', 'Denial', 'Official report', 'Surprise',
       'Joking'],
      dtype='object')

In [ ]:
# rearranging columns
train_df = train_df[['Tweet', 'Optimistic', 'Thankful', 'Empathetic', 'Pessimistic',
       'Anxious', 'Sad', 'Annoyed', 'Denial', 'Official report', 'Surprise',
       'Joking']]

In [ ]:
train_df

,Tweet,Optimistic,Thankful,Empathetic,Pessimistic,Anxious,Sad,Annoyed,Denial,Official report,Surprise,Joking
8863,Un puto tren por hora y todavía quiere ser put...,0,0,0,0,0,0,1,0,0,0,0
1889,Abril deseo: ser mejor dada la pérdida saludab...,1,0,1,0,0,0,0,0,0,0,0
3276,hijo de puta mi salud mental nunca ha sido tan...,0,0,0,0,1,0,1,0,0,0,0
2582,"Dios por la subasta, pero si la Corona empeora...",0,0,0,0,1,0,0,0,0,0,0
240,Se debería haber tenido una carta si theyve un...,0,0,1,0,0,0,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
4884,El acetaminofeno (Tylenol) * * algo ABT ibupro...,0,0,0,0,0,1,0,0,1,1,1
8525,PUTAS DE MIERDA Extraño el GYM usted. FUCK YOU...,0,0,0,0,0,0,1,0,0,0,0
3925,"JumiaHeroes Buenos días kenianos, que las prec...",1,0,0,0,1,0,0,0,0,1,0
856,Sí tengo c- virus de la corona cancelado o- GT...,0,0,0,0,0,0,0,0,0,0,1


In [ ]:
target_list = ['Optimistic', 'Thankful', 'Empathetic', 'Pessimistic',
       'Anxious', 'Sad', 'Annoyed', 'Denial', 'Official report', 'Surprise',
       'Joking']

In [ ]:
# hyperparameters
MAX_LEN = 256
TRAIN_BATCH_SIZE = 32
VALID_BATCH_SIZE = 32
EPOCHS = 2
LEARNING_RATE = 1e-05

In [ ]:

from transformers import BertForMaskedLM,BertTokenizer, BertModel

In [ ]:
tokenizer = BertTokenizer.from_pretrained("pytorch/", do_lower_case=False)


In [ ]:
class CustomDataset(torch.utils.data.Dataset):

    def __init__(self, df, tokenizer, max_len):
        self.tokenizer = tokenizer
        self.df = df
        self.title = df['Tweet']
        # print(self.df[target_list].values)
        self.targets = self.df[target_list].values
        self.max_len = max_len

    def __len__(self):
        return len(self.title)

    def __getitem__(self, index):
        title = str(self.title[index])
        title = " ".join(title.split())

        inputs = self.tokenizer.encode_plus(
            title,
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        # print(self.targets[index])
        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'token_type_ids': inputs["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.targets[index])
        }

In [ ]:
train_size = 0.7

val_df =train_df.copy()
train_df = train_df.sample(frac=train_size, random_state=200)
val_df = val_df.drop(train_df.index).reset_index(drop=True)
train_df = train_df.reset_index(drop=True)
train_df.shape, val_df.shape

((6300, 12), (2700, 12))

In [ ]:
train_dataset = CustomDataset(train_df, tokenizer, MAX_LEN)
valid_dataset = CustomDataset(val_df, tokenizer, MAX_LEN)

In [ ]:
train_data_loader = torch.utils.data.DataLoader(train_dataset, 
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset, 
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [ ]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [ ]:
device

device(type='cuda')

In [ ]:
def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into       
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss 
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [ ]:
class BERTClass(torch.nn.Module):
    def __init__(self):
        super(BERTClass, self).__init__()
        self.bert_model = BertModel.from_pretrained("pytorch/", return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 11)
    
    def forward(self, input_ids, attn_mask, token_type_ids):
        output = self.bert_model(
            input_ids, 
            attention_mask=attn_mask, 
            token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.pooler_output)
        output = self.linear(output_dropout)
        return output

model = BERTClass()
model.to(device)

Some weights of the model checkpoint at pytorch/ were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERTClass(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tr

In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
val_targets=[]
val_outputs=[]

In [ ]:
def train_model(n_epochs, training_loader, validation_loader, model, 
                optimizer, checkpoint_path, best_model_path):
   
  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf
   
 
  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
        #print('yyy epoch', batch_idx)
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)
        token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
        targets = data['targets'].to(device, dtype = torch.float)

        outputs = model(ids, mask, token_type_ids)

        optimizer.zero_grad()
        loss = loss_fn(outputs, targets)
        #if batch_idx%5000==0:
         #   print(f'Epoch: {epoch}, Training Loss:  {loss.item()}')
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        #print('before loss data in training', loss.item(), train_loss)
        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))
        print('after loss data in training', loss.item(), train_loss)
    
    print('############# Epoch {}: Training End     #############'.format(epoch))
    
    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################    
    # validate the model #
    ######################
 
    model.eval()
   
    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)
            token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, token_type_ids)

            loss = loss_fn(outputs, targets)
            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))
            val_targets.extend(targets.cpu().detach().numpy().tolist())
            val_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      # calculate average losses
      #print('before cal avg train loss', train_loss)
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      # print training/validation statistics 
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch, 
            train_loss,
            valid_loss
            ))
      
      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }
        
        # save checkpoint
      save_ckp(checkpoint, False, checkpoint_path, best_model_path)
        
      ## TODO: save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model

In [ ]:

ckpt_path = "/content/drive/MyDrive/Recuperacion de la informacion/BERT/multi-label/curr_ckpt"
best_model_path = "/content/drive/MyDrive/Recuperacion de la informacion/BERT/multi-label/best_model.pt"

In [ ]:
trained_model = train_model(EPOCHS, train_data_loader, val_data_loader, model, optimizer, ckpt_path, best_model_path)
# train_data_loader

############# Epoch 1: Training Start   #############
after loss data in training 0.6977332830429077 0.6977332830429077
after loss data in training 0.7015740275382996 0.6996536552906036
after loss data in training 0.6711342930793762 0.6901472012201945
after loss data in training 0.6751757264137268 0.6864043325185776
after loss data in training 0.6448654532432556 0.6780965566635132
after loss data in training 0.6360986232757568 0.6710969010988872
after loss data in training 0.6262750029563904 0.6646937727928163
after loss data in training 0.6160970330238342 0.6586191803216935
after loss data in training 0.6113699078559875 0.6533692611588373
after loss data in training 0.5812953114509583 0.6461618661880494
after loss data in training 0.571284294128418 0.6393548141826283
after loss data in training 0.5657535195350647 0.633221372961998
after loss data in training 0.5573841333389282 0.6273877391448388
after loss data in training 0.5600057244300842 0.6225747380937849
after loss data in train

In [ ]:
# testing
example = test_df.iloc[1]['Tweet']
print(example)
print(test_df.iloc[0])
encodings = tokenizer.encode_plus(
    example,
    None,
    add_special_tokens=True,
    max_length=MAX_LEN,
    padding='max_length',
    return_token_type_ids=True,
    truncation=True,
    return_attention_mask=True,
    return_tensors='pt'
)
model.eval()
with torch.no_grad():
    input_ids = encodings['input_ids'].to(device, dtype=torch.long)
    attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
    token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
    output = model(input_ids, attention_mask, token_type_ids)
    final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
    
    print(np.concatenate((np.array([train_df.columns[1:]]),final_output),axis=0))
    print('data')
    print(final_output)
    print()
    print(np.argsort(final_output, axis=1))
    print()
    print((1/(1 + np.exp(-1 * np.array(final_output)))))
    print()
    print()
    print(train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))])

Estimado Gracias por estar ahí para mí durante la mitad de la crisis coronavirus. Amor, tu mayor fan
ID                                             1245208599929987072.0
Tweet              cualquier otra persona que espera en China par...
Optimistic                                                         0
Thankful                                                           0
Empathetic                                                         0
Pessimistic                                                        0
Anxious                                                            0
Sad                                                                0
Annoyed                                                            0
Denial                                                             0
Official report                                                    0
Surprise                                                           0
Joking                                                             1
Na

In [ ]:
# add validation data  
df_clean_data = pd.read_excel('/content/df_final_clean.xlsx')

df_clean_data

,Unnamed: 0,datetime,tweet,tweet_stop_word,clean
0,0,2020-01-04 09:57:06,No es hora de lamentos y tampoco de críticas.H...,no es hora de lamentos tampoco de críticas hay...,no hora lamentos tampoco críticas hecho real c...
1,1,2020-01-05 12:56:25,@EleccionesCong Quiere aparentar distanciamien...,eleccionescong quiere aparentar distanciamien...,eleccionescong quiere aparentar distanciamient...
2,2,2020-01-05 21:32:35,@5ojgonzalez Maduro miente una vez más. Eligió...,ojgonzalez maduro miente una vez más eligió r...,ojgonzalez maduro miente vez eligió repetir gu...
3,3,2020-01-11 08:23:26,Creo que en este instante hay como una epidemi...,creo que en este instante hay como una epidemi...,creo instante epidemia diarrea ciudad siento m...
4,4,2020-01-15 20:09:51,@Cecilia605040 Se contagió del virus Arbizu,cecilia se contagió del virus arbizu,cecilia contagió virus arbizu
...,...,...,...,...,...
41182,41184,2021-12-30 21:01:43,"Estupefacto, pedí hablar con un médico y me at...",estupefacto pedí hablar con un médico me atend...,estupefacto pedí hablar médico atendió señorit...
41183,41185,2021-12-30 21:01:47,"Cuánta gente se contagiará, quedará con secuel...",cuánta gente se contagiará quedará con secuela...,cuánta gente contagiará quedará secuelas morir...
41184,41186,2021-12-30 22:17:13,@patriciagamarra Hay empresas que saben que su...,patriciagamarra hay empresas que saben que su...,patriciagamarra empresas saben personal covid ...
41185,41187,2021-12-30 22:43:44,"Quiero torta de chocolate, puedo comer torta d...",quiero torta de chocolate puedo comer torta de...,quiero torta chocolate puedo comer torta choco...


In [ ]:
df_clean_data.isna().sum()

Unnamed: 0         0
datetime           0
tweet              0
tweet_stop_word    0
clean              0
dtype: int64

In [ ]:
# testing
def label_data_single(df):
  size = df.shape[0]
  labels = np.zeros((size,), dtype=np.int)
  names = np.chararray((size,), itemsize=20)
  for i in range(size):

    text = df.iloc[i]['tweet_stop_word']

    encodings = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=MAX_LEN,
        padding='max_length',
        return_token_type_ids=True,
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    model.eval()

    with torch.no_grad():
        input_ids = encodings['input_ids'].to(device, dtype=torch.long)
        attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
        token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
        output = model(input_ids, attention_mask, token_type_ids)
        final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
        
        index = int(np.argmax(final_output, axis=1))
        name = train_df.columns[1:].to_list()[int(np.argmax(final_output, axis=1))]

        labels[i] = index
        names[i] = name

  df['label'] = labels
  df['name'] = names

  return df

In [ ]:
df_clean_data[df_clean_data["tweet_stop_word"].isna()].to_csv('nan_rows.csv')
df_clean_data.sum()

Unnamed: 0         41189
Unnamed: 0.1       41189
datetime           41189
tweet_original     41189
tweet_stop_word    40914
tweet_clean        40889
hashtag             9564
dtype: int64

In [ ]:
df_1 = df_clean_data.copy()
# df_1
label_data_single(df_1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  after removing the cwd from sys.path.


,Unnamed: 0,datetime,tweet,tweet_stop_word,clean,label,name
0,0,2020-01-04 09:57:06,No es hora de lamentos y tampoco de críticas.H...,no es hora de lamentos tampoco de críticas hay...,no hora lamentos tampoco críticas hecho real c...,6,b'Annoyed'
1,1,2020-01-05 12:56:25,@EleccionesCong Quiere aparentar distanciamien...,eleccionescong quiere aparentar distanciamien...,eleccionescong quiere aparentar distanciamient...,6,b'Annoyed'
2,2,2020-01-05 21:32:35,@5ojgonzalez Maduro miente una vez más. Eligió...,ojgonzalez maduro miente una vez más eligió r...,ojgonzalez maduro miente vez eligió repetir gu...,6,b'Annoyed'
3,3,2020-01-11 08:23:26,Creo que en este instante hay como una epidemi...,creo que en este instante hay como una epidemi...,creo instante epidemia diarrea ciudad siento m...,10,b'Joking'
4,4,2020-01-15 20:09:51,@Cecilia605040 Se contagió del virus Arbizu,cecilia se contagió del virus arbizu,cecilia contagió virus arbizu,10,b'Joking'
...,...,...,...,...,...,...,...
41182,41184,2021-12-30 21:01:43,"Estupefacto, pedí hablar con un médico y me at...",estupefacto pedí hablar con un médico me atend...,estupefacto pedí hablar médico atendió señorit...,10,b'Joking'
41183,41185,2021-12-30 21:01:47,"Cuánta gente se contagiará, quedará con secuel...",cuánta gente se contagiará quedará con secuela...,cuánta gente contagiará quedará secuelas morir...,0,b'Optimistic'
41184,41186,2021-12-30 22:17:13,@patriciagamarra Hay empresas que saben que su...,patriciagamarra hay empresas que saben que su...,patriciagamarra empresas saben personal covid ...,10,b'Joking'
41185,41187,2021-12-30 22:43:44,"Quiero torta de chocolate, puedo comer torta d...",quiero torta de chocolate puedo comer torta de...,quiero torta chocolate puedo comer torta choco...,10,b'Joking'


In [ ]:
df_1.to_excel('df_labeled_final.xlsx')


In [ ]:
df_2 = df_clean_data.copy()
label_data(df_2)

,Unnamed: 0,Date,tweet,label,name
0,0,2021-12-30 23:46:08+00:00,enserio con toda esto del covid solo deseo ade...,6,Annoyed
1,1,2021-12-30 22:43:44+00:00,quiero torta de chocolate puedo comer torta de...,10,Joking
2,2,2021-12-30 22:17:13+00:00,patriciagamarra hay empresas que saben que su ...,10,Joking
3,3,2021-12-30 21:01:47+00:00,cuánta gente se contagiará quedará con secuela...,0,Optimistic
4,4,2021-12-30 21:01:43+00:00,estupefacto pedí hablar con un médico atendió ...,10,Joking
...,...,...,...,...,...
22964,22964,2020-02-15 18:54:10+00:00,china pone en cuarentena los billetes usados p...,8,Official report
22965,22965,2020-02-15 02:40:38+00:00,febrero coronavirus nivel globalcovid infectad...,8,Official report
22966,22966,2020-02-13 13:48:34+00:00,nace una nueva enfermedad “ covid ” http co ge...,10,Joking
22967,22967,2020-02-12 03:36:41+00:00,sobre corona virus coronavirus fuente ops de f...,8,Official report


In [ ]:
# testing
def label_data_multi(df):
  size = df.shape[0]
  labels = []
  names = []
  for i in range(10):

    text = df.iloc[i]['tweet']

    encodings = tokenizer.encode_plus(
        text,
        None,
        add_special_tokens=True,
        max_length=MAX_LEN,
        padding='max_length',
        return_token_type_ids=True,
        truncation=True,
        return_attention_mask=True,
        return_tensors='pt'
    )
    
    model.eval()

    with torch.no_grad():
        input_ids = encodings['input_ids'].to(device, dtype=torch.long)
        attention_mask = encodings['attention_mask'].to(device, dtype=torch.long)
        token_type_ids = encodings['token_type_ids'].to(device, dtype=torch.long)
        output = model(input_ids, attention_mask, token_type_ids)
        final_output = torch.sigmoid(output).cpu().detach().numpy().tolist()
        
        min = np.min(final_output[0])
        max = np.max(final_output[0])

        normalization = (final_output[0] - min)/(max - min)

        print()
        print(normalization)
        print()

        index = np.argsort(final_output[0], axis=0)[::-1][:3]

        print(index)
        print(train_df.columns[1:])

  return df

In [ ]:
df_11 = df_clean_data.copy()
label_data_multi(df_11)


[1.         0.17245483 0.09324784 0.13964621 0.23631182 0.18523865
 0.13777579 0.         0.09576996 0.23688321 0.28781122]

[ 0 10  9]
Index(['Optimistic', 'Thankful', 'Empathetic', 'Pessimistic', 'Anxious', 'Sad',
       'Annoyed', 'Denial', 'Official report', 'Surprise', 'Joking'],
      dtype='object')

[1.         0.30658673 0.12486706 0.0923748  0.0953255  0.11123908
 0.1143002  0.         0.02861364 0.21184434 0.44553637]

[ 0 10  1]
Index(['Optimistic', 'Thankful', 'Empathetic', 'Pessimistic', 'Anxious', 'Sad',
       'Annoyed', 'Denial', 'Official report', 'Surprise', 'Joking'],
      dtype='object')

[0.27609449 0.00791835 0.         0.12392968 0.14575073 0.37345108
 0.39397994 0.01832848 0.0058464  0.12651684 1.        ]

[10  6  5]
Index(['Optimistic', 'Thankful', 'Empathetic', 'Pessimistic', 'Anxious', 'Sad',
       'Annoyed', 'Denial', 'Official report', 'Surprise', 'Joking'],
      dtype='object')

[0.31519736 0.03459523 0.         0.16873684 0.15960697 0.46989919
 0.34

,Unnamed: 0,datetime,tweet
0,0,2020-01-06 11:13:36,essaludperu algún día cambiará habrá mayor per...
1,1,2020-01-22 16:12:39,caleons jajaja tmre estos tuits alegran el día...
2,2,2020-01-26 02:07:56,lo que mete gonzaleszela el trajín voluntad de...
3,3,2020-01-27 13:34:12,llegó el coronavirus cusco
4,4,2020-01-27 16:23:16,ya hay posibles portadores del coronavirus en ...
...,...,...,...
2083,2083,2020-12-25 17:38:22,jaajaj fuck creo que tuve covid
2084,2084,2020-12-26 06:30:44,policías murieron en todo el país por causa de...
2085,2085,2020-12-29 14:04:47,qué onda con todos los geis que están en rio d...
2086,2086,2020-12-30 07:17:17,vizcarra el peor gestor durante la pandemia
